# Primeiro exemplo

Exemplo de código para análise de imagens do PAN-STARRS1

Este foi o primeiro teste desenvolvido para visualização das capacidades do código. Ferramentas melhores estão sendo desenvolvidas em paralelo.

In [ ]:
from astropy.coordinates import SkyCoord
from astropy.io import fits
import astropy.units as u
from astropy.time import Time
from astropy.wcs import WCS
import numpy as np
from astroquery.imcce import Skybot
from astroquery.vizier import Vizier
import glob

In [ ]:
lista = glob('*.fits')
columns = ['RA_ICRS', 'DE_ICRS', 'pmRA', 'pmDE']
epoch = Time('J2016.0', scale='tdb')

In [ ]:
def create_region(image_name):
    # Lê o header da imagem 
    header = fits.getheader(image_name)
    time = header['DATE-OBS']
    exp = header['EXPTIME']
    naxis1 = int(header['NAXIS1'])
    naxis2 = int(header['NAXIS2'])
    cdelt = header['CDELT1']*u.deg/u.pix

    obstime = Time(time) + exp*u.s/2 # Calcula o instante da imagem
    w = WCS(header[0:30])  # Lê o World Coordinate System da imagem

    b1 = w.pixel_to_world(0,0)
    b2 = w.pixel_to_world(naxis1, naxis2)
    center = w.pixel_to_world(naxis1/2, naxis2/2)
    sep = b1.separation(b2)
    print(sep)

    w.wcs.crpix += np.array([57, -59])  # Offset manual para as imagens de teste

    # Faz a busca pelos asteroides conhecidos no SkyBot
    results = Skybot.cone_search(center, sep/1.8, obstime)
    asteroides = SkyCoord(ra=results['RA'], dec=results['DEC'])
    px, py = w.world_to_pixel(asteroides)
    px = np.array(px, ndmin=1)
    py = np.array(py, ndmin=1)
    unc = results['posunc']

    # Faz a busca pelas estrelas conhecidas no Vizier
    vquery = Vizier(timeout=600, row_limit=-1, columns=columns)
    result = vquery.query_region(center, radius=sep/1.8, catalog='I/355/gaiadr3', cache=False)[0]
    pmra = np.nan_to_num(result['pmRA'], 0)
    pmdec = np.nan_to_num(result['pmDE'], 0)
    stars = SkyCoord(ra=result['RA_ICRS'], dec=result['DE_ICRS'], pm_ra_cosdec=pmra, pm_dec=pmdec, obstime=epoch)
    obs_stars = stars.apply_space_motion(new_obstime=obstime)
    pxs, pys = w.world_to_pixel(obs_stars)
    pxs = np.array(pxs, ndmin=1)
    pys = np.array(pys, ndmin=1)

    # Escreve o arquivo .reg que contêm as localizações das estrelas e dos asteroides.
    f = open(image_name[:-4] + 'reg', 'w')
    for i in range(len(px)):
        f.write('image; circle( {}, {}, {}) # color=red width=2 text={{{}}}\n'.format(px[i], py[i], 5, results['Name'][i]))
        f.write('image; circle( {}, {}, {}) # color=red width=2 dash=1\n'.format(px[i], py[i], (unc[i]/cdelt).decompose().value))
    for i in range(len(pxs)):
        f.write('image; circle( {}, {}, {}) # color=green width=2\n'.format(pxs[i], pys[i], 5))
    f.close()

In [ ]:
for l in lista:
    print(l)
    create_region(l)

Para visualizar a imagem com as regiões, é necessário utilizar o DS9, com o seguinte comando

ds9 {nome_da_imagem}.fits -region {nome_da_imagem}.reg

Caso tenha mais de uma imagem, basta fazer:

ds9 {nome_da_imagem1}.fits -region {nome_da_imagem1}.reg {nome_da_imagem2}.fits -region {nome_da_imagem2}.reg {nome_da_imagem3}.fits -region {nome_da_imagem3}.reg